# wordslab-notebooks-lib.chat

> Chat with local and remote LLMs in the context of the wordslab-notebooks environment 

In [ ]:
#| default_exp chat

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
# import ...